# Indicator Derivation

Since the full edge gadget is probably too much for anyone to really process, here we break out the derivation of each component indicator set. This should allow us to communicate what exactly we're doing here better.

### Initialization Code

In [2]:
%matplotlib widget

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import ipywidgets as widgets

from scipy.interpolate import CubicSpline
from scipy.ndimage.filters import gaussian_filter1d

# data directory
dataDir = "./data/"

### Color Map Specification

In [3]:
# color maps -- note convert from data specification to labels
cmap_label = ['Traditional', 'Jet', 'Kindlmann', 'Gray']
cmap_str = ['Default', 'Jet', 'Kindlmann', 'Gray']

# load color maps, and make accessible by label
cmaps = dict(zip(cmap_label,list(pd.read_csv(dataDir+v.lower()+".csv") for v in cmap_str)))

### Indicator Specification

In [4]:
param_label = ['L*', 'C*', 'h']
indicator_label = ['Cusps (High Curvature)', 'Inflection Points']

# dictionary for switching settings
# returns (sigma, gradient_threshold, curvature_threshold)
indicator_settings = {
    'Traditional': {
        'L*': {
            'Cusps (High Curvature)': (0, None, 90000),
            'Inflection Points': (0.6, 295, None)
        },
        'C*': {
            'Cusps (High Curvature)': (0, None, 37000),
            'Inflection Points': (0.8, 135, None)
        },
        'h': {
            'Cusps (High Curvature)': (0, None, None),
            'Inflection Points': (0, 3, None)
        }
    },
    'Jet': {
        'L*': {
            'Cusps (High Curvature)': (0, None, None),
            'Inflection Points': (0, None, None)
        },
        'C*': {
            'Cusps (High Curvature)': (0, None, None),
            'Inflection Points': (0, None, None)
        },
        'h': {
            'Cusps (High Curvature)': (0, None, None),
            'Inflection Points': (0, None, None)
        }
    },
    'Kindlmann': {
        'L*': {
            'Cusps (High Curvature)': (0, None, None),
            'Inflection Points': (0, None, None)
        },
        'C*': {
            'Cusps (High Curvature)': (0, None, None),
            'Inflection Points': (0, None, None)
        },
        'h': {
            'Cusps (High Curvature)': (0, None, None),
            'Inflection Points': (0, None, None)
        }
    },
    'Gray': {
        'L*': {
            'Cusps (High Curvature)': (0, None, None),
            'Inflection Points': (0, None, None)
        },
        'C*': {
            'Cusps (High Curvature)': (0, None, None),
            'Inflection Points': (0, None, None)
        },
        'h': {
            'Cusps (High Curvature)': (0, None, None),
            'Inflection Points': (0, None, None)
        }
    }
}

### Code for Individual Plots

In [5]:
# Import Convenience Classes
from ConvenienceClasses import SingleLinePlotHandle, Interpolated, ParamControl

def plotParam(figure, colormap, param='L*', xs=np.linspace(0,1,10000)):
    # setup gridspecs
    gs = gridspec.GridSpec(4, 1, height_ratios=[1.5,4,4,4])
    
    # switch dicts
    colormap_switch = {'L*': colormap.L, 'C*':colormap.C, 'h':colormap.h}
    ylims_switch = {'L*':[0,100], 'C*':[0,140], 'h':[0,2*np.pi]}
    
    # generate spline
    current_param = Interpolated(colormap.x, colormap_switch[param], 0.5)
    
    # create image
    rgb = colormap[['r','g','b']].values
    rgb.shape = (1,rgb.shape[0],rgb.shape[1])
    rgb_image = np.repeat(rgb, 50, axis=0)
    
    axes = []
    # plot image
    ax = plt.subplot(gs[0,0])
    im_obj = ax.imshow(rgb_image, interpolation='nearest', aspect='auto')
    ax.get_yaxis().set_visible(False)
    ax.tick_params(labelbottom=False)
    axes.append(ax)
    
    # plot spline
    ax = plt.subplot(gs[1,0])
    l, = ax.plot(xs, current_param.spline(xs), color='C0')
    ax.set_xlim([0,1])
    ax.tick_params(labelbottom=False)
    ax.set_ylim(ylims_switch[param])
    if (param is 'h'):
        ax.set_yticks([0, np.pi/2, np.pi, 3*np.pi/2, 2*np.pi])
        ax.set_yticklabels(['$0$', r'$\frac{\pi}{2}$', r'$\pi$', r'$\frac{3\pi}{2}$', r'$2\pi$'])
    current_param.handles.append(SingleLinePlotHandle(ax, l))
    axes.append(ax)
    
    # plot |grad|
    ax = plt.subplot(gs[2,0])
    l, = ax.plot(xs, np.abs(current_param.grad(xs)), color='C0')
    ax.set_xlim([0,1])
    ax.tick_params(labelbottom=False)
    current_param.handles.append(SingleLinePlotHandle(ax, l))
    axes.append(ax)
    
    # plot |curv|
    ax = plt.subplot(gs[3,0])
    l, = ax.plot(xs, np.abs(current_param.curv(xs)), color='C0')
    ax.set_xlim([0,1])
    current_param.handles.append(SingleLinePlotHandle(ax, l))
    axes.append(ax)
    
    return (axes, current_param, im_obj)

### Interactive Plot

In [6]:
from matplotlib.ticker import AutoLocator, ScalarFormatter

plt.ioff() # turns interactive mode off
plt.clf()  # clears current figure

# ------------ INITIALIZE CONTROLS, PT. 1 ------------

# initialize controls
cmap_select = widgets.Select(
                                options=cmap_label,
                                value='Traditional',
                                rows=4,
                                description='Color Map:',
                                disabled=False
                             )

param_select = widgets.Select(
                                        options=param_label,
                                        value='L*',
                                        rows=3,
                                        description='Channel:',
                                        disabled=False
                             )

indicator_select = widgets.Select(
                                        options=indicator_label,
                                        value='Cusps (High Curvature)',
                                        rows=2,
                                        description='Indicators:',
                                        disabled=False
                             )

reset_button = widgets.Button(description='reset')

# ------------ SETUP FIGURE ------------

# setup figure
fig_edgePlot = plt.figure(figsize=(3.5,5))# 11,4
fig_edgePlot.subplots_adjust(top=0.95, right=0.95, left=0.17, bottom=0.05)

xs = np.linspace(0,1,10000) # values for plotting functions

# plot
axes, current_param, im_obj = plotParam(fig_edgePlot, cmaps[cmap_select.value], param_select.value, xs=xs)

# fig_edgePlot.show() will display a static plot, for interactive see below

# ------------ INITIALIZE CONTROLS, PT. 2 ------------

current_control = ParamControl(current_param, axes[1], axes[2], axes[3])

# ------------ UPDATE SCHEMES ------------
    
def update(figure, xs=[], control=None, grad=False, curv=False): 
        
    if control is not None: 
        if   grad: control.update_grad()
        elif curv: control.update_curv()
        else:      control.update_full(xs)
        
        figure.canvas.draw_idle()
        figure.canvas.flush_events()
        
def override(figure, axes, colormap, param_str, controls, im_obj=None):
    
    # override data
    param_switch = {'L*': colormap.L, 'C*':colormap.C, 'h':colormap.h}
    current_param.override(colormap.x, param_switch[param_str], current_param.sigma)
    
    # override ylims and ticks
    ylims_switch = {'L*':[0,100], 'C*':[0,140], 'h':[0,2*np.pi]}
    axes[1].set_ylim(ylims_switch[param_str])
    if (param_str == 'h'):
        axes[1].set_yticks([0, np.pi/2, np.pi, 3*np.pi/2, 2*np.pi])
        axes[1].set_yticklabels(['$0$', r'$\frac{\pi}{2}$', r'$\pi$', r'$\frac{3\pi}{2}$', r'$2\pi$'])
    else:
        axes[1].yaxis.set_major_locator(AutoLocator())
        axes[1].yaxis.set_major_formatter(ScalarFormatter())
        
    # override indicator colors
    colors_switch = {'L*':['#0E6089','#5E50A3'], 'C*':['#0E6089', '#DB7723'], 'h':['#992B65', '#1C9B37']}
    cc = colors_switch[param_str]
    controls.setColors(cc[0], cc[1])
    
    # re-draw
    update(figure, xs=xs, control=controls)

def update_cmap(change, im_obj):
    
    # create new image
    colormap = cmaps[cmap_select.value]
    
    # create new image
    rgb = colormap[['r','g','b']].values
    rgb.shape = (1,rgb.shape[0],rgb.shape[1])
    rgb_image = np.repeat(rgb, 50, axis=0)
    im_obj.set_data(rgb_image)
    
    # override plots
    override(fig_edgePlot, axes, colormap, param_select.value, current_control)
    
    #reset indicators
    update_indicators(current_control)  # indicator_select.value = None

def update_param(change):
    
    # override plots
    override(fig_edgePlot, axes, cmaps[cmap_select.value], param_select.value, current_control)
    
    # reset indicators
    update_indicators(current_control)  # indicator_select.value = None

def update_indicators(controls):
    
    s, g_thresh, c_thresh = indicator_settings[cmap_select.value][param_select.value][indicator_select.value]

    controls.sigma_control.value = s
    controls.gradient_threshold.value = g_thresh if g_thresh is not None else controls.gradient_threshold.max
    controls.curvature_threshold.value = c_thresh if c_thresh is not None else controls.curvature_threshold.max

# ------------ LINK CONTROLS ------------

current_control.sigma_control.observe(       lambda change : update(fig_edgePlot, control=current_control, xs=xs),     names='value' )
current_control.curvature_threshold.observe( lambda change : update(fig_edgePlot, control=current_control, curv=True), names='value' )
current_control.gradient_threshold.observe(  lambda change : update(fig_edgePlot, control=current_control, grad=True), names='value' )

cmap_select.observe( lambda change : update_cmap(change, im_obj), names='value')
param_select.observe(update_param, names='value')
indicator_select.observe(lambda change : update_indicators(current_control), names='value')

reset_button.on_click(lambda b : update_indicators(current_control))

# ------------ DISPLAY ------------

# update initial controls state
update_indicators(current_control)

# display
widgets.HBox([widgets.VBox([cmap_select,param_select,indicator_select,current_control.widget,reset_button]), fig_edgePlot.canvas])

#### TODO -- Properly Handle NaN Roots

There's a runtime warning issued by the thesholding command when the profile is (appx.) linear, becuase the SciPy UnivateSpline root finding routine is returning NaNs. ConveienceClasses.py has been updated to remove the NaNs from the processing, but we should determine and handle the actual root of the probelm...

In [7]:
# EXAMPLE -- Generates Runtime Warning
cmap_select.value = 'Kindlmann'
param_select.value = 'L*'
update_indicators(current_control)

temp_param = current_control._param
temp_param._curv_roots[(np.abs(temp_param._curv(temp_param._curv_roots)) > current_control.curvature_threshold.value)]
# temp_param._spline.roots(discontinuity=False)

/Users/samquinan/Development/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in greater
  import sys


array([], dtype=float64)